# Linear models
These codes are taken from linear models page
https://bashtage.github.io/linearmodels/doc/panel/examples/examples.html

In [1]:
from linearmodels.datasets import wage_panel
import pandas as pd
data = wage_panel.load()
year = pd.Categorical(data.year)
data = data.set_index(['nr', 'year'])  # multiindex
data['year'] = year                    # add year column (will create automatic year dummies)
print(wage_panel.DESCR)
print(data.head())


F. Vella and M. Verbeek (1998), "Whose Wages Do Unions Raise? A Dynamic Model
of Unionism and Wage Rate Determination for Young Men," Journal of Applied
Econometrics 13, 163-183.

nr                       person identifier
year                     1980 to 1987
black                    =1 if black
exper                    labor market experience
hisp                     =1 if Hispanic
hours                    annual hours worked
married                  =1 if married
educ                     years of schooling
union                    =1 if in union
lwage                    log(wage)
expersq                  exper^2
occupation               Occupation code

         black  exper  hisp  hours  married  educ  union     lwage  expersq  \
nr year                                                                       
13 1980      0      1     0   2672        0    14      0  1.197540        1   
   1981      0      2     0   2320        0    14      1  1.853060        4   
   1982      0    

In [ ]:
Note that that has to be multiindex: entity/date

In [2]:
data

black  exper  hisp  hours  married  educ  union     lwage  \
nr    year                                                              
13    1980      0      1     0   2672        0    14      0  1.197540   
      1981      0      2     0   2320        0    14      1  1.853060   
      1982      0      3     0   2940        0    14      0  1.344462   
      1983      0      4     0   2960        0    14      0  1.433213   
      1984      0      5     0   3071        0    14      0  1.568125   
...           ...    ...   ...    ...      ...   ...    ...       ...   
12548 1983      0      8     0   2080        1     9      0  1.591879   
      1984      0      9     0   2080        1     9      1  1.212543   
      1985      0     10     0   2080        1     9      0  1.765962   
      1986      0     11     0   2080        1     9      1  1.745894   
      1987      0     12     0   3380        1     9      1  1.466543   

            expersq  occupation  year  
nr    year                             
13    1980        1           9  1980  
      1981        4           9  1981  
      1982        9           9  1982  
      1983       16           9  1983  
      1984       25           5  1984  
...             ...         ...   ...  
12548 1983       64           5  1983  
      1984       81           5  1984  
      1985      100           5  1985  
      1986      121           5  1986  
      1987      144           5  1987  

[4360 rows x 11 columns]

## Basic regression
Note add constant to exogenous variables


In [3]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm
exog_vars = ['black','hisp','exper','expersq','married', 'educ', 'union', 'year']
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(data.lwage, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.1893
Estimator:                  PooledOLS   R-squared (Between):              0.2066
No. Observations:                4360   R-squared (Within):               0.1692
Date:                Sun, May 03 2020   R-squared (Overall):              0.1893
Time:                        11:18:29   Log-likelihood                   -2982.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      72.459
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             72.459
                            

## Models with additional terms

### Additional term uncorrelated with regressors

#### Random effects


In [4]:
from linearmodels.panel import RandomEffects
mod = RandomEffects(data.lwage, exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:              RandomEffects   R-squared (Between):              0.1853
No. Observations:                4360   R-squared (Within):               0.1799
Date:                Sun, May 03 2020   R-squared (Overall):              0.1828
Time:                        11:21:12   Log-likelihood                   -1622.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      68.409
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             68.409
                            

The model fit is fairly similar, although the return to experience has changed substantially, as has its significance. This is partially explainable by the inclusion of the year dummies which will fit the trend in experience and so only the cross-sectional differences matter. The quasi-differencing in the random effects estimator depends on a quantity that depends on the relative variance of the idiosyncratic shock and the common shock. This can be accessed using variance_decomposition.

In [5]:
re_res.variance_decomposition

Effects                   0.106946
Residual                  0.123324
Percent due to Effects    0.464438
Name: Variance Decomposition, dtype: float64

The coefficient θi
 determines how much demeaning takes place. When this value is 1, the RE model reduces to the pooled model since this occurs when there is no variance in the effects. When panels are unbalanced it will vary across entities, but in this balanced panel all values are the same.

In [6]:
re_res.theta.head()

,theta
nr,
13,0.645059
17,0.645059
18,0.645059
45,0.645059
110,0.645059


### Additional term uncorrelated with regressors
When effects are correlated with the regressors the RE and BE estimators are not consistent. The usual solution is to use Fixed Effects which are available in PanelOLS. Fixed effects are called entity_effects when applied to entities and time_effects when applied to the time dimension:

### Fixed effects: time invariant
Entity effects are included by setting entity_effects=True. This is equivalent to including dummies for each entity. In this panel, this would add 545 dummy variables and estimation of the model would be considerably slower. PanelOLS does not actually use dummy variables and instead uses group-wise demeaning to achieve the same effect.

In [8]:
from linearmodels.panel import PanelOLS
exog_vars = ['expersq', 'union', 'married', 'year']
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True)
fe_res = mod.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:                   PanelOLS   R-squared (Between):             -0.0052
No. Observations:                4360   R-squared (Within):               0.1806
Date:                Sun, May 03 2020   R-squared (Overall):              0.0807
Time:                        11:31:15   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      83.851
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             83.851
                            

In [10]:
# Check what happens when you dont drop collinear exp
from linearmodels.panel import PanelOLS
exog_vars = ['exper','expersq', 'union', 'married', 'year']
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True)
fe_res = mod.fit()
print(fe_res)

AbsorbingEffectError: 
The model cannot be estimated. The included effects have fully absorbed
one or more of the variables. This occurs when one or more of the dependent
variable is perfectly explained using the effects included in the model.

The following variables or variable combinations have been fully absorbed
or have become perfectly collinear after effects are removed:

          const, exper, expersq, union, married, year.1981, year.1982, year.1983, year.1984, year.1985, year.1986, year.1987

Set drop_absorbed=True to automatically drop absorbed variables.


In [11]:
# Check what happens when you dont drop collinear exp

########Answer: messy!!! ############ drops 1987....
from linearmodels.panel import PanelOLS
exog_vars = ['exper','expersq', 'union', 'married', 'year']
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True, drop_absorbed=True)
fe_res = mod.fit()
print(fe_res)




                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:                   PanelOLS   R-squared (Between):             -0.0528
No. Observations:                4360   R-squared (Within):               0.1806
Date:                Sun, May 03 2020   R-squared (Overall):              0.0552
Time:                        11:33:26   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      83.851
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             83.851
                            

/Users/pelly/opt/miniconda3/envs/geo_plotnine/lib/python3.8/site-packages/linearmodels/panel/model.py:1667: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

year.1987

  warnings.warn(


In [12]:
# Check what happens when you dont drop collinear exp

########Answer: changing order, exper is the one droped......

from linearmodels.panel import PanelOLS
exog_vars = ['expersq', 'union', 'married', 'year', 'exper']
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True, drop_absorbed=True)
fe_res = mod.fit()
print(fe_res)




                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:                   PanelOLS   R-squared (Between):             -0.0052
No. Observations:                4360   R-squared (Within):               0.1806
Date:                Sun, May 03 2020   R-squared (Overall):              0.0807
Time:                        11:34:38   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      83.851
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             83.851
                            

/Users/pelly/opt/miniconda3/envs/geo_plotnine/lib/python3.8/site-packages/linearmodels/panel/model.py:1667: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

exper

  warnings.warn(


### Time and entity fixed effects

In [14]:
from linearmodels.panel import PanelOLS
exog_vars = ['expersq','union','married']
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.lwage, exog, entity_effects=True, time_effects=True)
fe_te_res = mod.fit()
print(fe_te_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.0216
Estimator:                   PanelOLS   R-squared (Between):             -0.0052
No. Observations:                4360   R-squared (Within):              -0.4809
Date:                Sun, May 03 2020   R-squared (Overall):             -0.2253
Time:                        11:40:34   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      27.959
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(3,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             27.959
                            

# First differences
First differencing is an alternative to using fixed effects when there might be correlation. When using first differences, time-invariant variables must be excluded. Additionally, only one linear time-trending variable can be included since this will look like a constant. This variable will soak up all time-trends in the data, and so interpretations of these variable can be challenging.

In [15]:
from linearmodels.panel import FirstDifferenceOLS
exog_vars = ['exper','expersq', 'union', 'married']
exog = data[exog_vars]
mod = FirstDifferenceOLS(data.lwage, exog)
fd_res = mod.fit()
print(fd_res)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                  lwage   R-squared:                        0.0268
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.5491
No. Observations:                3815   R-squared (Within):               0.1763
Date:                Sun, May 03 2020   R-squared (Overall):              0.5328
Time:                        11:40:56   Log-likelihood                   -2305.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      26.208
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(4,3811)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             26.208
                            

## Compare models
Model results can be compared using compare. compare accepts lists of results, a dictionary of results where the key is interpreted as the model name.

In [17]:
from linearmodels.panel import compare
print(compare({'RE':re_res,'Pooled':pooled_res}))

                 Model Comparison                 
                                   RE       Pooled
--------------------------------------------------
Dep. Variable                   lwage        lwage
Estimator               RandomEffects    PooledOLS
No. Observations                 4360         4360
Cov. Est.                  Unadjusted   Unadjusted
R-squared                      0.1806       0.1893
R-Squared (Within)             0.1799       0.1692
R-Squared (Between)            0.1853       0.2066
R-Squared (Overall)            0.1828       0.1893
F-statistic                    68.409       72.459
P-value (F-stat)               0.0000       0.0000
===================== =============== ============
const                          0.0234       0.0921
                             (0.1546)     (1.1761)
black                         -0.1394      -0.1392
                            (-2.9054)    (-5.9049)
hisp                           0.0217       0.0160
                             (0

## Heteroskedasticity Robust Covariance

White’s robust covariance can be used by setting cov_type='robust. This estimator adds some robustness against certain types of specification issues but should not be used when using fixed effects (entity effects) since it is no longer robust. Instead a clustered covariance is required.

In [18]:
exog_vars = ['black','hisp','exper','expersq','married', 'educ', 'union']
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(data.lwage, exog)
robust = mod.fit(cov_type='robust')

## Clustered errors
The usual variable to cluster are are entity or entity and time. The can be implemented using cov_type='clustered' and the additional keyword arguments cluster_entity=True and/or cluster_time=True.



In [19]:
clust_entity = mod.fit(cov_type='clustered', cluster_entity=True)
clust_entity_time = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)
from collections import OrderedDict
res = OrderedDict()
res['Robust'] = robust
res['Entity'] = clust_entity
res['Entity-Time'] = clust_entity_time
print(compare(res))

                     Model Comparison                    
                           Robust      Entity Entity-Time
---------------------------------------------------------
Dep. Variable               lwage       lwage       lwage
Estimator               PooledOLS   PooledOLS   PooledOLS
No. Observations             4360        4360        4360
Cov. Est.                  Robust   Clustered   Clustered
R-squared                  0.1866      0.1866      0.1866
R-Squared (Within)         0.1679      0.1679      0.1679
R-Squared (Between)        0.2027      0.2027      0.2027
R-Squared (Overall)        0.1866      0.1866      0.1866
F-statistic                142.61      142.61      142.61
P-value (F-stat)           0.0000      0.0000      0.0000
===================== =========== =========== ===========
const                     -0.0347     -0.0347     -0.0347
                        (-0.5360)   (-0.2892)   (-0.3145)
black                     -0.1438     -0.1438     -0.1438
              

In [21]:
### Other clusters not entity or time 
#Be aware, small number of clusters are not advisable
clust_entity = mod.fit(cov_type='clustered', clusters=data.occupation)
print(data.occupation.value_counts())
print(clust_entity)

5    934
6    881
9    509
4    486
1    453
7    401
2    399
3    233
8     64
Name: occupation, dtype: int64
                          PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.1866
Estimator:                  PooledOLS   R-squared (Between):              0.2027
No. Observations:                4360   R-squared (Within):               0.1679
Date:                Sun, May 03 2020   R-squared (Overall):              0.1866
Time:                        11:46:57   Log-likelihood                   -2989.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      142.61
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(7,4352)
Min Obs:                       8.0000                                         

## Using 'patsy' formulas

In [22]:
from statsmodels.datasets import grunfeld
data = grunfeld.load_pandas().data
data = data.set_index(['firm','year'])
print(data.head())

                       invest   value  capital
firm           year                           
General Motors 1935.0   317.6  3078.5      2.8
               1936.0   391.8  4661.7     52.6
               1937.0   410.6  5387.1    156.9
               1938.0   257.7  2792.2    209.2
               1939.0   330.8  4313.2    203.4


In [23]:
from linearmodels import PanelOLS
mod = PanelOLS.from_formula('invest ~ value + capital + EntityEffects', data=data)
print(mod.fit())

                          PanelOLS Estimation Summary                           
Dep. Variable:                 invest   R-squared:                        0.7667
Estimator:                   PanelOLS   R-squared (Between):              0.8223
No. Observations:                 220   R-squared (Within):               0.7667
Date:                Sun, May 03 2020   R-squared (Overall):              0.8132
Time:                        11:54:32   Log-likelihood                   -1167.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      340.08
Entities:                          11   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,207)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             340.08
                            

In [24]:
mod = PanelOLS.from_formula('invest ~ 1 + value + capital + EntityEffects', data=data)
print(mod.fit())

                          PanelOLS Estimation Summary                           
Dep. Variable:                 invest   R-squared:                        0.7667
Estimator:                   PanelOLS   R-squared (Between):              0.8193
No. Observations:                 220   R-squared (Within):               0.7667
Date:                Sun, May 03 2020   R-squared (Overall):              0.8071
Time:                        11:54:44   Log-likelihood                   -1167.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      340.08
Entities:                          11   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,207)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             340.08
                            

In [25]:
mod = PanelOLS.from_formula('invest ~ 1 + value + capital + EntityEffects + TimeEffects', data=data)
print(mod.fit())

                          PanelOLS Estimation Summary                           
Dep. Variable:                 invest   R-squared:                        0.7253
Estimator:                   PanelOLS   R-squared (Between):              0.7944
No. Observations:                 220   R-squared (Within):               0.7566
Date:                Sun, May 03 2020   R-squared (Overall):              0.7856
Time:                        11:55:01   Log-likelihood                   -1153.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      248.15
Entities:                          11   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,188)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             248.15
                            

In [26]:
from linearmodels import BetweenOLS, FirstDifferenceOLS, PooledOLS
mod = BetweenOLS.from_formula('invest ~ 1 + value + capital', data=data)
print(mod.fit())

                         BetweenOLS Estimation Summary                          
Dep. Variable:                 invest   R-squared:                        0.8644
Estimator:                 BetweenOLS   R-squared (Between):              0.8644
No. Observations:                  11   R-squared (Within):               0.4195
Date:                Sun, May 03 2020   R-squared (Overall):              0.7616
Time:                        11:55:20   Log-likelihood                   -61.997
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      25.500
Entities:                          11   P-value                           0.0003
Avg Obs:                       20.000   Distribution:                     F(2,8)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             25.500
                            

In [27]:
mod = PooledOLS.from_formula('invest ~ 1 + value + capital', data=data)
print(mod.fit())

                          PooledOLS Estimation Summary                          
Dep. Variable:                 invest   R-squared:                        0.8179
Estimator:                  PooledOLS   R-squared (Between):              0.8426
No. Observations:                 220   R-squared (Within):               0.7357
Date:                Sun, May 03 2020   R-squared (Overall):              0.8179
Time:                        11:55:29   Log-likelihood                   -1301.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      487.28
Entities:                          11   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,217)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             487.28
                            